In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/Data/Cleaned_Null_Values+Sale_Data+Feature_Engineering.csv')
df

,Unnamed: 0.1,Unnamed: 0,cost,bedrooms,bathrooms,sqft_lot,location,zip_code,house_lot,last_sold(months),ready_to_build,total_house_size,total_rooms
0,0,0,105000.0,3.0,2.0,5227.2,137,601.0,920.0000,0,False,6.147200e+03,5.0
1,1,1,80000.0,4.0,2.0,3484.8,137,601.0,1527.0000,0,False,5.011800e+03,6.0
2,2,2,67000.0,2.0,1.0,6534.0,13201,795.0,748.0000,0,False,7.282000e+03,3.0
3,3,3,145000.0,4.0,2.0,4356.0,21140,731.0,1800.0000,0,False,6.156000e+03,6.0
4,4,4,65000.0,6.0,2.0,2178.0,16317,680.0,2714.4713,0,False,4.892471e+03,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1411580,1411580,1414368,299000.0,3.0,2.0,29185.2,13490,99901.0,2714.4713,0,False,3.189967e+04,5.0
1411581,1411581,1414369,299000.0,3.0,2.0,29185.2,13490,99901.0,2714.4713,0,False,3.189967e+04,5.0
1411582,1411582,1414370,199000.0,3.0,2.0,57063.6,13490,99901.0,2714.4713,0,False,5.977807e+04,5.0
1411583,1411583,1414371,1575000.0,3.0,2.0,12744784.8,13490,99901.0,2714.4713,0,False,1.274750e+07,5.0


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sc = StandardScaler()
X = sc.fit_transform(df.drop(['cost','Unnamed: 0','Unnamed: 0.1'],axis=1))
y = sc.fit_transform(df[['cost']])


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [31]:
import pickle
with open('D:/Client_Projects/Sir_Malik/Price-Prediction-Model_v2/Data_Cleaning & Feature Engineering/pipelines/scaler.pkl','wb') as f:
    pickle.dump(sc,f)

## ANN Impelementation

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# 🔹 Confirm GPU
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("TensorFlow is using:", "GPU" if tf.config.list_physical_devices('GPU') else "CPU")

# 🔹 Enable XLA JIT compiler (accelerates training on GPU)
tf.config.optimizer.set_jit(True)

# 🔹 Enable mixed precision (great for RTX & newer GPUs)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

# Split train/validation
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)


def build_ann(input_dim, task="regression", optimizer="adam"):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(64, activation="relu"))

    # Output + loss + metrics
    if task == "binary":
        model.add(layers.Dense(1, activation="sigmoid", dtype="float32"))
        loss = "binary_crossentropy"
        metrics = ["accuracy", keras.metrics.AUC(name="auc")]
    elif task == "multi":
        model.add(layers.Dense(len(set(y_train)), activation="softmax", dtype="float32"))
        loss = "sparse_categorical_crossentropy"
        metrics = ["accuracy"]
    else:  # regression
        model.add(layers.Dense(1, activation="linear", dtype="float32"))
        loss = "mse"
        metrics = ["mae", "mse"]

    # 🔹 Choose optimizer
    optimizers = {
        "adam": keras.optimizers.Adam(learning_rate=1e-3),
        "nadam": keras.optimizers.Nadam(learning_rate=1e-3),
        "sgd": keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, nesterov=True),
        "rmsprop": keras.optimizers.RMSprop(learning_rate=1e-3)
    }
    opt = optimizers.get(optimizer, keras.optimizers.Adam(1e-3))

    # Compile
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    return model


# Build model (choose optimizer: "adam", "nadam", "sgd", "rmsprop")
model = build_ann(X_train.shape[1], task="regression", optimizer="adam")

# Callbacks
early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
checkpoint = keras.callbacks.ModelCheckpoint("best_ann.h5", monitor="val_loss", save_best_only=True)

# 🔹 Make dataset pipeline for GPU efficiency
train_dataset = tf.data.Dataset.from_tensor_slices((X_tr, y_tr)).batch(4096).prefetch(tf.data.AUTOTUNE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(4096).prefetch(tf.data.AUTOTUNE)

# 🔹 Train with GPU acceleration
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

# Evaluate
results = model.evaluate(val_dataset, verbose=1)
print("Evaluation:", dict(zip(model.metrics_names, results)))


In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

In [10]:
model = Sequential([Dense(2048,activation='relu',input_shape=(X_train.shape[1],)),
                    Dense(1024, activation='relu'),
                    Dense(512, activation='relu'),
                    Dense(256, activation='relu'),
                    Dense(128, activation='relu'),
                    Dense(1, activation='linear',dtype='float32')])

d:\Client_Projects\Sir_Malik\Price-Prediction-Model_v2\.venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 2048)           │        22,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,809,857 (10.72 MB)

 Trainable params: 2,809,857 (10.72 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
import os
os.makedirs('Data_Cleaning & Feature Engineering/models',exist_ok=True)
opt = Adam(learning_rate=0.01)
early_stop = EarlyStopping(monitor='val_loss',patience=15,restore_best_weights=True)
model_checkpoint = ModelCheckpoint('models/best_model.h5',monitor='val_loss',save_best_only=True)
tb = TensorBoard(log_dir='logs')

In [14]:
model.compile(optimizer=opt,
            loss=['mse'],
            metrics=['accuracy','mae','mse'])

In [18]:
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[early_stop,model_checkpoint,tb],batch_size=256)

Epoch 1/100
4411/4412 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.0000e+00 - loss: 1.5153 - mae: 0.2149 - mse: 1.5153

4412/4412 ━━━━━━━━━━━━━━━━━━━━ 287s 65ms/step - accuracy: 0.0000e+00 - loss: 1.9883 - mae: 0.2161 - mse: 1.9883 - val_accuracy: 0.0000e+00 - val_loss: 0.4574 - val_mae: 0.2203 - val_mse: 0.4574
Epoch 2/100
4412/4412 ━━━━━━━━━━━━━━━━━━━━ 316s 72ms/step - accuracy: 0.0000e+00 - loss: 1.0209 - mae: 0.2104 - mse: 1.0209 - val_accuracy: 0.0000e+00 - val_loss: 0.4815 - val_mae: 0.1911 - val_mse: 0.4815
Epoch 3/100
4411/4412 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.0000e+00 - loss: 0.9762 - mae: 0.2082 - mse: 0.9762

4412/4412 ━━━━━━━━━━━━━━━━━━━━ 295s 67ms/step - accuracy: 0.0000e+00 - loss: 1.0375 - mae: 0.2151 - mse: 1.0375 - val_accuracy: 0.0000e+00 - val_loss: 0.4513 - val_mae: 0.1929 - val_mse: 0.4513
Epoch 4/100
4412/4412 ━━━━━━━━━━━━━━━━━━━━ 345s 72ms/step - accuracy: 0.0000e+00 - loss: 1.0395 - mae: 0.2138 - mse: 1.0395 - val_accuracy: 0.0000e+00 - val_loss: 0.4828 - val_mae: 0.1917 - val_mse: 0.4828
Epoch 5/100
4412/4412 ━━━━━━━━━━━━━━━━━━━━ 310s 70ms/step - accuracy: 0.0000e+00 - loss: 1.0545 - mae: 0.2171 - mse: 1.0545 - val_accuracy: 0.0000e+00 - val_loss: 0.4873 - val_mae: 0.2074 - val_mse: 0.4873
Epoch 6/100
4412/4412 ━━━━━━━━━━━━━━━━━━━━ 297s 67ms/step - accuracy: 0.0000e+00 - loss: 1.0420 - mae: 0.2107 - mse: 1.0420 - val_accuracy: 0.0000e+00 - val_loss: 0.4841 - val_mae: 0.1985 - val_mse: 0.4841
Epoch 7/100
4412/4412 ━━━━━━━━━━━━━━━━━━━━ 306s 69ms/step - accuracy: 0.0000e+00 - loss: 1.0432 - mae: 0.2125 - mse: 1.0432 - val_accuracy: 0.0000e+00 - val_loss: 0.4834 - val_mae: 0.1906 

In [19]:
model.save('models/best_model.keras')

In [29]:
model.predict(X_test[[3]])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step


array([[-0.16183078]], dtype=float32)

In [30]:
y_test[3]

array([-0.232842])